# Grain Boundary Generation
GBMaker is a python package designed to help with the creation of grain boundaries.
It contains two main structure classes and two generator classes.

## Generator Classes
The generator classes provide a simple way to create either of the two structure classes.
Being generators they are designed to be iterated over however, contain a method to return a `list` of their contents. 
### GrainBoundaryGenerator
The GrainBoundaryGenerator is a class that builds an iterator of symmetrically inequivalent terminations of Grains in a GrainBoundary.
It is initialised with the required information to build a GrainBoundary and has a single method, `get_grain_boundaries()`, that is equivalent to `list(GrainBoundaryGenerator)`.
It is the simplest and often most efficient way to create `GrainBoundaries` as it will call the `GrainGenerator` and can handle the creation of different `Grains` either side of the boundary.

In [ ]:
from gbmaker import GrainBoundaryGenerator
# To see further help uncomment the line below
# help(GrainBoundaryGenerator)

#### Initialisation
The GrainBoundaryGenerator can be initialised from any bulk cell and will attempt to build the grains using a primitive verison of the cell with the miller index relative to the standard conventional unit cell.
There are currently two ways of doing this:
1) Initialising the class with a pymategen.core.Structure

In [ ]:
from pymatgen.core import Structure
bulk = Structure(
    [[4.175, 0, 0], [0, 4.175, 0], [0, 0, 4.175]], 
    ["Mg", "Mg", "O", "O"], 
    [[0, 0, 0], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0, 0, 0.5]],
)
gbg = GrainBoundaryGenerator(
    bulk, 
    [3, 1, 0],
    mirror_z=True, 
    translation_vec=[0,0,2.0],
    bulk_repeats=3,
    orthogonal_c=True,
)

2) Reading the structure from a file 

In [ ]:
gbg = GrainBoundaryGenerator.from_file(
    "bulk-POSCAR", 
    [3, 1, 0], 
    mirror_z=True, 
    translation_vec=[0,0,2.0],
    bulk_repeats=3,
    orthogonal_c=True,
)
grain_boundaries = gbg.get_grain_boundaries()

Notice the warning returned. This is because the bulk cell supplied here is not the same as the conventional unit cell.
## Important
The GrainBoundaryGenerator will build the grains using a primitive cell as a way to try and reduce the size of the outputted Structure. **However the miller index is relative to the standard conventional unit cell.** This means the cell supplied is not always the cell that the miller indices are relative to. 

From here grain boundaries can be built using the get_grain_boundaries() method.
And visualised using any library that interfaces with pymatgen.

In [ ]:
import nglview as nv

struct = grain_boundaries[0].get_structure()

view = nv.show_pymatgen(struct)
view.clear()
view.add_ball_and_stick(assembly="UNITCELL")
view.add_unitcell()
view

### Arguments for Initialising GrainBoundaryGenerator
#### `bulk_0: Structure`
This this the bulk cell to be analysed for finding the conventional and primitive cell. 
Unless this is the conventional cell a warning will be shown displaying the conventional cell that was found. 
#### `miller_0: ArrayLike`
This is the miller index of the `Grain` and is relative to the conventional cell found by the symmetry analyser.
#### `bulk_1: Optional[Structure] = None`
An optional second `Structure` can be passed as the bulk cell to be used for the second grain. 
This is useful for creating interfaces between materials.
**Currently there is no checking for commensurate lattices.**
Lattices will be made as orthogonal as possible without increasing the number of atoms in the oriented unit cell.
Further matching must be done by the user.
The second `Grain` is scaled such that its a and b fractional are the same in the first `Grain`'s lattice whilst maintaining its own spacing in the Cartesian z-direction.
#### `miller_1: Optional[ArrayLike] = None`
An optional second index can be supplied for the second `Grain`.
This is useful for creating aymmetric `GrainBoundries`.
Like with the optional second bulk `Structure` there is no lattice matching attempted and the user is responsible for making the lattices commensurate.
#### `mirror_x: bool = False`
The second `Grain` can be mirrored along any of the 3 Cartesian directions, this flag will mirror it along the x-direction if set to `True`.
#### `mirror_y: bool = False`
The second `Grain` can be mirrored along any of the 3 Cartesian directions, this flag will mirror it along the y-direction if set to `True`.
#### `mirror_z: bool = False`
The second `Grain` can be mirrored along any of the 3 Cartesian directions, this flag will mirror it along the z-direction if set to `True`. 
This is most commonly used for creating twinned `GrainBoundaries`.
#### `vacuum: Optional[float] = None`
Incases where the `Grains` are asymmetric the `GrainBoundaries` can have two disinct interfaces, in this case a vacuum gap can be inserted between them so that the there is only one interface and two surfaces.
When no vacuum is supplied the z-component of the `translation_vec` is used to seperate both interfaces.
#### `translation_vec: ArrayLike = [0.0, 0.0, 0.0]`
This vector is applied to the second `Grain` to shift it relative to the first. 
It is in Cartesian coordinates and has the unit Angstrom.
#### `merge_tol: Optional[float] = None`
An optional tolerance, in Angstrom, for merging atoms at the boundary together.
If used this will often raise a warning as the site properties of the merged atoms will be removed if they are different. 
This is okay to ignore. 
#### `reconstruction: Optional[Callable[["Grain", Site], bool]] = None`
An optional function that excepts a `Grain` and `Site` and returns a `True` or `False` value on whether that site should be included in the structure.
An example of this is provided in examples/MgO/11-2_grain_boundary_reconstruction.py.
#### `ftol: float = 0.1`
A tolerance for calculating possible shifts in the c-direction of the oriented unit cell.
#### `tol: float = 0.1`
A symmetry tolerance for matching `Structures` so that `Grains` with the same termintion are only generated once.
#### `max_broken_bonds: int = 0`
How many broken bonds, from [bonds](#bonds:-Optional[Dict[Sequence[SpeciesLike],-float]]-=-None), are allowed before the `Grain` is either repaired or discarded.
#### `bonds: Optional[Dict[Sequence[SpeciesLike], float]] = None`
An optional dictionary describing maximum bond lengths between pairs of atoms. 
If this is passed along with `repair` then surfaces will be "repaired" by moving atoms from one side of the grain to the other to try to satisfy the bulk coordination of the atoms.
#### `repair: bool = False`
If the [max_broken_bonds](#max_broken_bonds:-int-=-0) is violated then should we attempt to repair the surface.
#### `symmetrize: bool = False`
A flag on whether only symmetric `Grain` terminations should be considered.
If the `Grain` is asymmetric then we should try and make it symmetric by removing atoms from the surface, this results in a non-stoichiometric `Grain`.
If symmetric and stoichiometric `Grains` are required then the `GrainBoundaryGenerator` can be filtered to exclude the `Grains` with the property `symmetrize = True`.
#### `bulk_repeats: int = 1`
There are three ways to set the thickness of the `Grains` in the `GrainBoundary` and only one is considered.
The hierarchy of these arguments is as follows:
1) [hkl_thickness](#hkl_thickness:-Optional[float]-=-None)
2) [thickness](#thickness:-Optional[float]-=-None)
3) [bulk_repeats](#bulk_repeats:-int-=-1)

This method will set the amount of periodic repeats of the oriented unit cell to use in both `Grains`.
#### `thickness: Optional[float] = None`
There are three ways to set the thickness of the `Grains` in the `GrainBoundary` and only one is considered.
The hierarchy of these arguments is as follows:
1) [hkl_thickness](#hkl_thickness:-Optional[float]-=-None)
2) [thickness](#thickness:-Optional[float]-=-None)
3) [bulk_repeats](#bulk_repeats:-int-=-1)

This method will set the minimum required thickness ensure that both `Grains` are atleast as thick as the supplied thickness in Angstrom.
#### `hkl_thickness: Optional[float] = None`
There are three ways to set the thickness of the `Grains` in the `GrainBoundary` and only one is considered.
The hierarchy of these arguments is as follows:
1) [hkl_thickness](#hkl_thickness:-Optional[float]-=-None)
2) [thickness](#thickness:-Optional[float]-=-None)
3) [bulk_repeats](#bulk_repeats:-int-=-1)

This method will set the minimum required thickness ensure that both `Grains` are atleast as thick as the supplied thickness in hkl units.
#### `orthogonal_c: bool = False`
This flag will set the c-vector to be aligned along the z-axis.
This is useful for ensuring that the boundaries between the `Grains` are symmetric in terms of relative position.

The arguments for the class method `from_file` are identical however the filenames for the bulk structures are supplied rather than the `Structure`. 

In [ ]:
gbg = GrainBoundaryGenerator.from_file(
    filename_0="bulk-POSCAR", 
    miller_0=[1, 1, 1],
    filename_1=None,
    miller_1=None,
    mirror_x=False,
    mirror_y=False,
    mirror_z=True, 
    vacuum=None,
    translation_vec=[0,0,0],
    merge_tol=0.1,
    reconstruction=None,
    ftol=0.1,
    tol=0.1,
    max_broken_bonds=0,
    bonds=None,
    repair=False,
    symmetrize=True,
    bulk_repeats=3,
    thickness=None,
    hkl_thickness=None,
    orthogonal_c=True,
)
gb_list = gbg.get_grain_boundaries()
for gb in gb_list:
    gb.ab_supercell([2, 2])

top_view = nv.show_pymatgen(gb_list[0].get_sorted_structure())
top_view.clear()
top_view.add_ball_and_stick(assembly="UNITCELL")
top_view.add_unitcell()
bottom_view = nv.show_pymatgen(gb_list[1].get_sorted_structure())
bottom_view.clear()
bottom_view.add_ball_and_stick(assembly="UNITCELL")
bottom_view.add_unitcell()
import ipywidgets as ipw
ipw.VBox([top_view, bottom_view])

### Using the GrainBoundaryGenerator


## Structure Classes
The structure classes contain the relevant information required to build either a single grain or a grain boundary.
### Grain

In [ ]:
from gbmaker import Grain
# To see further help uncomment the line below
# help(Grain)

Initialising a Grain is often done from the specialised generator class.
However the class has its own class method for initialising from an oriented unit cell.
It is recommended to initialise using this method over constructing the grain itself as the oriented unit cell has special requirements that are ensured by this method.

In [ ]:
from pymatgen.core import Structure
ouc = Structure.from_file("./ouc_POSCAR")
print(ouc)

In [ ]:
grain = Grain.from_oriented_unit_cell(ouc, [3, 1, 0], 0)
print(grain.oriented_unit_cell)